In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from datasets import load_dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import evaluate
import time
import nltk
import numpy as np

nltk.download('wordnet')
rouge = evaluate.load('rouge')

/Users/jinhyunpark/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jinhyunpark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Function to generate the output from the model
def generate_output(model, tokenizer, example):
    inputs = tokenizer.encode(example, return_tensors="pt")
    outputs = model.generate(inputs, max_new_tokens=np.inf)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [3]:
def calculate_rouge(true_sentence, predicted_sentence):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(true_sentence, predicted_sentence)
    return scores

In [4]:
def calculate_bleu(true_sentence, predicted_sentence):
    true_tokens = true_sentence.split()
    predicted_tokens = predicted_sentence.split()
    smoothie = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], predicted_tokens, smoothing_function=smoothie)
    return bleu_score

### Datasets

In [5]:
# english to X is only possible for T5
from datasets import load_dataset, load_from_disk

# ds_de_en = load_dataset("wmt/wmt14", "de-en")
# ds_fr_en = load_dataset("wmt/wmt15", "fr-en")
# ds_ro_en = load_dataset("wmt/wmt16", "ro-en")

# ds_de_en.save_to_disk("../wmt14_de_en")
# ds_fr_en.save_to_disk("../wmt15_fr_en")
# ds_ro_en.save_to_disk("../wmt16_ro_en")

ds_de_en = load_from_disk("../wmt14_de_en")
ds_fr_en = load_from_disk("../wmt15_fr_en")
ds_ro_en = load_from_disk("../wmt16_ro_en")

### Models

In [6]:
t5_tokenizer_small = AutoTokenizer.from_pretrained('t5-small')
t5_model_small = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
print("Done with small")

t5_tokenizer_base = AutoTokenizer.from_pretrained('t5-base')
t5_model_base = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
print("Done with base")

Done with small
Done with base


### Generate translations

In [7]:
for i in range(5):
    print(f"------------ Sample {i+1} ------------")
    
    input_text = f"translate English to German: {ds_de_en['train'][i]['translation']['en']}"
    true_translation = ds_de_en['train'][i]['translation']['de']
    predicted_translation = generate_output(model=t5_model_small, tokenizer=t5_tokenizer_small, example=input_text)

    true_tokens = t5_tokenizer_small.tokenize(true_translation)
    predicted_tokens = t5_tokenizer_small.tokenize(predicted_translation)

    print("Input:", input_text)
    print("True Translation:", true_translation)
    print("Predicted Translation:", predicted_translation)
    
    # 1. METEOR
    print("METEOR score:", meteor_score([true_tokens], predicted_tokens))
    # 2. ROUGE
    rouge_scores = calculate_rouge(true_translation, predicted_translation)
    print("ROUGE scores:", rouge_scores)
    # 3. BLEU
    bleu_score = calculate_bleu(true_translation, predicted_translation)
    print("BLEU score:", bleu_score)

    print(" ")

------------ Sample 1 ------------
Input: translate English to German: Resumption of the session
True Translation: Wiederaufnahme der Sitzungsperiode
Predicted Translation: Wiederaufnahme der Sitzungsperiode
METEOR score: 0.9993141289437586
ROUGE scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
BLEU score: 0.5623413251903491
 
------------ Sample 2 ------------
Input: translate English to German: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
True Translation: Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.
Predicted Translation